In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 10 14:50:01 2016

@author: Duanhong
"""
import numpy as np
import scipy.stats as ss
import pandas as pd
import math 
import matplotlib.pyplot as plt
import datetime as dt
import operator

#----- Option Delta Calculation -----------#
def Option_Delta(opt_type, S0, K, r, sigma, T):
        """opt_type: put or call """
        d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T)/(sigma * np.sqrt(T))
        if opt_type=="Call":
            return(ss.norm.cdf(d1))
        else:
            return(-ss.norm.cdf(-d1))

#----- Compute the delta value for options -------#


class Portfolio1(object):
    """ An US Stock we randomly choose. Stocks have the
    following properties:
    
    Attributes:
        name: A string representing the stock's name.
        price: Adj Close price representing the stock's price 
                     from 12/5/1996 to 12/5/2016 .
        date: Ranging from 12/5/1996 to 12/5/2016
    """
    
    def __init__(self, name, price, date):
        """Return a stock object whose name is *name* and close
        price is *price*."""
        self.name = name
        self.price = price
        self.date = date
                
    #-----Type I VaR model   
    def Historic_VaR(self, windowlen, horizon, p, S0):
        """ Return a stock object's Historic VaR after setting 
        a stock VaR's parameters, e.g. windowlen(2,5,10 years),
        horizon = 5/252, p = 0.99, portfolio value S0 = 10000. """
        A_hist_log_rtn = []
        A_hist_log_rtn_sq = []
        for i in range(1, len(self.price)- 5):
            hist_log_return = math.log( self.price[i]/self.price[i+5] )
            A_hist_log_rtn.append(hist_log_return)
            A_hist_log_rtn_sq.append(hist_log_return**2)         
        # Calculate Historic VaR
        hist_VaR = []        
        for i in range(len(self.price)-252*windowlen):
            his_VaR_list = sorted(A_hist_log_rtn[i:i+252*windowlen])
            his_VaR_val = (-S0) * his_VaR_list[13]
            hist_VaR.append(his_VaR_val)    
        return(hist_VaR)
  
    #------Type IIa VaR model
    def GBM_VaR(self, windowlen, horizon, p, S0):
        """ Return a stock object's Historic VaR after setting 
        a stock VaR's parameters, e.g. windowlen(2,5,10 years),
        horizon = 5/252, p = 0.99, portfolio value S0 = 10000. """
        log_rtn = []
        log_rtn_sq = []
        for i in range(1 ,len(self.price)-1):
            log_return = math.log( self.price[i]/self.price[i+1] )
            log_rtn.append(log_return)
            log_rtn_sq.append(log_return**2) 
        # Calculate GBM VaR
        GBM_VaR = []   
        for i in range(len(self.price)-252*windowlen):
            vol_years = np.std(log_rtn[i:i+252*windowlen]) * np.sqrt(252)
            mu_years = np.mean(log_rtn[i:i+252*windowlen])*252 + (vol_years**2)/2
            GBM_VaR_val = S0 - S0 * np.exp(vol_years * horizon**(0.5)* ss.norm.ppf(1-p)+(mu_years - pow(vol_years,2)/2)*horizon)
            GBM_VaR.append(GBM_VaR_val)      
        return(GBM_VaR)
        
    #-----Type IIb VaR model   
    def Wgt_GBM_VaR(self, lambda_wgt, windowlen, horizon, p, S0):
        """wgt is the exponential weight value like 0.99^n. """
        list_lambda = []
        for i in range(len(self.price)):
            list_lambda.append(lambda_wgt**i)
            
        wgt_log_rtn = []
        wgt_log_rtn_sq = []
        for i in range(len(self.price)-1 ):
            log_return = math.log( self.price[i]/self.price[i+1] )
            wgt_log_return = log_return *  list_lambda[i] 
            wgt_log_rtn.append(wgt_log_return)
            log_return_sq = log_return ** 2
            wgt_log_rtn_sq.append( log_return_sq * list_lambda[i] ) 
        # Calculate Weighted VaR under GBM model
        wgt_VaR = []
        for j in range(len(self.price)-252*windowlen):
            wgt_mu_lambda0 =  sum(wgt_log_rtn[j:j+252*windowlen])/sum(list_lambda[j:j+252*windowlen])
            wgt_vol_lambda = np.sqrt(252) * np.sqrt(sum(wgt_log_rtn_sq[j:j+252*windowlen])/sum(list_lambda[j:j+252*windowlen])- wgt_mu_lambda0**2)
            wgt_mu_lambda =252 * wgt_mu_lambda0 + (wgt_vol_lambda**2)/2
            wgt_VaR_val = S0 - S0 * np.exp( wgt_vol_lambda * horizon **(0.5)* ss.norm.ppf(1-p) + (wgt_mu_lambda - pow(wgt_vol_lambda,2)/2)*horizon )        
            wgt_VaR.append(wgt_VaR_val)
        return(wgt_VaR)
       
    #-----Type III VaR model  
    def Monte_Carlo_VaR(self, windowlen, horizon, p, S0):
        """"  This is Monte Carlo VaR under GBM assumption  """
        log_rtn = []
        log_rtn_sq = []
        for i in range(1 ,len(self.price)-1):
            log_return = math.log(self.price[i]/self.price[i+1] )
            log_rtn.append(log_return)
            log_rtn_sq.append(log_return**2) 
        # Calculate GBM MC VaR   
        MC_VaR = []
        for i in range(len(self.price)-252*windowlen):
            vol_years = np.std(log_rtn[i:i+252*windowlen]) * np.sqrt(252)
            mu_years = np.mean(log_rtn[i:i+252*windowlen])*252 + (vol_years**2)/2            
            random = vol_years * ss.norm.ppf(np.random.rand())
            MC_VaR_val =S0-S0 * np.exp(vol_years*horizon**(0.5)*ss.norm.ppf(1-p)+(mu_years + random-pow(vol_years,2)/2)*horizon)
            MC_VaR.append(MC_VaR_val)  
        return(MC_VaR)
    
    #-----BackTest 
    def BackTest(self,VaR_list, windowlen, horizon, S0):
        """Back test the VaR_list(eg.GBM VaR) based VaR estimates for long and 
        short portfolios by counting the number of times the VaR on each date 
        is exceeded by the subsequent horizon(eg. 5/252 ) change in each 1 year window. """
        nexcep = [ ]
        for i in range(len(VaR_list)):
            window_data = self.price[(len(VaR_list)-i-252*windowlen):(len(VaR_list)-i-1)]
            excep = 0
            for j in range( len(window_data) - 5):
                share = S0/window_data[j]
                pricet = window_data[j+5]
                loss = S0 - pricet * share
                if loss > VaR_list[len(VaR_list)-i-252 + j]:
                    excep = excep + 1
                else:
                    excep = excep  
            nexcep.append(excep)        
        return(nexcep)
        
    

class Portfoio2(object):
    def __init__(self, name, price1, price2, share1, share2, date):
        """Return a stock object whose name is *name* and close
        price is *price*."""
        self.name = name
        self.price1 = price1
        self.price2 = price2
        self.share1 = share1
        self.share2 = share2
        self.date = date
       
    def Port_BM_VaR(self, windowlen, horizon, p, S0):
        """ share is a vector containing stocks' shares in a portfolio.  """
        A_log_rtn = []
        A_log_rtn_sq = []
        for i in range(1 , len(self.price1)-1):
            log_return = math.log(self.price1[i] / self.price1[i+1]) 
            A_log_rtn.append(log_return)
            A_log_rtn_sq.append(log_return**2) 
            
        I_log_rtn = []
        I_log_rtn_sq = []
        for i in range(1 , len(self.price1)-1):
            log_return = math.log(self.price2[i] /self.price2[i+1]) 
            I_log_rtn.append(log_return)
            I_log_rtn_sq.append(log_return**2) 
    
        A_vol_years = []
        A_mu_years = []  
        I_vol_years = []
        I_mu_years = []
        corr = []    
    
        for i in range(len(self.price1)-252 * windowlen):
             vol_years1= np.std(A_log_rtn[i:i+252*windowlen]) * np.sqrt(252)
             mu_years1 = np.mean(A_log_rtn[i:i+252*windowlen])*252 + (vol_years1**2)/2
        
             vol_years2= np.std(I_log_rtn[i:i+252*windowlen]) * np.sqrt(252)
             mu_years2 = np.mean(I_log_rtn[i:i+252*windowlen])*252 + (vol_years2**2)/2
        
             x = np.asarray(A_log_rtn[i:i+252*windowlen])
             y = np.asarray(I_log_rtn[i:i+252*windowlen])
             X = np.vstack((x, y))
             cov = np.cov(X)
             corr_val = cov[0,1] * 252 / (vol_years1 * vol_years2)
             A_vol_years.append(vol_years1)
             I_vol_years.append(vol_years2)
             A_mu_years.append(mu_years1)
             I_mu_years.append(mu_years2)
             corr.append(corr_val)
        
        ##################### Window VaR and ES ################
        Port_VaR = []

        for i in range(len(self.price1)-252*windowlen):
            sigma = np.matrix([A_vol_years[i], I_vol_years[i]])
            rho = np.matrix([[1, corr[i]],[corr[i],1]] )
    
            x0 = np.dot(sigma, rho).T
            cov_mat = np.dot(x0, sigma)
    
            s0 = np.matrix([ self.price1[i], self.price2[i] ])
            pos = np.matrix([self.share1, self.share2])
            IndexVal = np.dot(pos, s0.T)
            portPos = pos/IndexVal * S0

            two_sto_val_split = np.array([[portPos[0,0]*s0[0,0]],[portPos[0,1]*s0[0,1]]]) 
            E_V_t = two_sto_val_split[0,0] * np.exp(A_mu_years[i] * 5/252) + two_sto_val_split[1,0] * np.exp(I_mu_years[i] * 5/252)
    
            x4 = np.exp(cov_mat*5/252)
            x1 = np.matrix([np.exp(A_mu_years[i]*5/252), np.exp(I_mu_years[i]*5/252)])
            x2 = np.dot(two_sto_val_split, two_sto_val_split.T)
            x3 = np.matrix([[x4[0,0]*x2[0,0],x4[0,1]*x2[0,1] ] , [x4[1,0]*x2[1,0], x4[1,1]*x2[1,1]]])
    
            E_V_t_sq = x1 * x3 * x1.T
            std_V_t = np.sqrt( E_V_t_sq - E_V_t ** 2)
            VaR_val = sum(two_sto_val_split) - (E_V_t + ss.norm.ppf(1-p) * std_V_t)
            Port_VaR.append(VaR_val.item(0))
        
        return(Port_VaR)


        


In [2]:
###--------------Portfolio VaRs except BM VaR------------------###   
#------======= SET 1 =======---------#
path = '~/Desktop/stos_opts_data.csv'
st_opt =  pd.read_csv(path, header = 1) 
data = st_opt.values

date = list(data[:,0])
AMD_close = list(data[:,2])
MCD_close = list(data[:,10])
GS_close = list(data[:,14])
INTC_close = list(data[:,6])
INTC_vol = list(data[:,5])
GE_close = list(data[:,14])
GE_vol = list(data[:,13])

list1 = [x*3000 for x in AMD_close] 
list2 = [x*2000 for x in MCD_close] 
list3 = [x*(-500) for x in GS_close]
#----- Option Delta Calculation -------#
            

opt1_Delta = []
for i in range(len(INTC_close)):
    delta=Option_Delta('Call', INTC_close[i], INTC_close[i], 0.005, INTC_vol[i], 1)
    opt1_Delta.append(delta)
opt1_shares = 100
list4 = [x * delta * opt1_shares for x,delta in zip(INTC_close, opt1_Delta) ]


opt2_Delta = []
for i in range(len(INTC_close)):
    delta=Option_Delta('Call', GE_close[i],  GE_close[i], 0.005, GE_vol[i], 1)
    opt2_Delta.append(delta)
    
opt2_shares = -50
list5 = [x * delta * opt2_shares for x,delta in zip(GE_close, opt2_Delta) ]

portfolio1 = [sum(x) for x in zip(list1, list2, list3, list4, list5)]
port1 = Portfolio1('portfolio1',portfolio1, date)  



In [3]:
#####Calculate Various VaR Models
Hist_VAR_port1 = port1.Historic_VaR(5, 5/252, 0.99, 10000)
GBM_VAR_port1 = port1.GBM_VaR(5, 5/252, 0.99, 10000)
Wgt_GBM_VAR_port1 = port1.Wgt_GBM_VaR(0.999, 5, 5/252, 0.99, 10000)
MC_VAR_port1 = port1.Monte_Carlo_VaR(5, 5/252, 0.99, 10000)
Hist_backtest1 = port1.BackTest(Hist_VAR_port1, 1, 5, 10000)



In [6]:
########## Plot figures
#------Historic VaR Figure
timeline = data[1:252*15,0]
timeline = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in timeline]
fig, ax = plt.subplots()
ax.plot(timeline, Hist_VAR_port1[1:252*15],'b-', label='Port1 historic VaR 5 year')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Port1 historic VaR 5 year')
plt.show()
##-----(MC) GBM VaR Figure
fig, ax = plt.subplots()
ax.plot(timeline, MC_VAR_port1[1:252*15],'b-', label='Port1 Monte Carlo VaR 5 year')
ax.plot(timeline, GBM_VAR_port1[1:252*15],'r-', label='Port1 GBM VaR 5 year')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Portfolio1 GBM & MC GBM VaR')
plt.show()
##------wgt GBM VaR Figure
fig, ax = plt.subplots()
ax.plot(timeline, Wgt_GBM_VAR_port1[1:252*15],'r-', label='lambda = 0.999')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Port1 weighted GBM VaR 5 year')
plt.show()
##-----Historic Backtest Figure
fig, ax = plt.subplots()
ax.plot(timeline, Hist_backtest1[1:252*15],'r-', label='Port1 Backtest')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Port1 Historic VaR BackTest 5 year')
plt.show()




In [5]:
###-------------Portfolio BM VaR-----------------###

AMD_close = list(data[:,2])
INTC_close = list(data[:,6])
MCD_close = list(data[:,10])

port = Portfoio2('port1',AMD_close,MCD_close, 1000, 200, data[:,0] )
BM_port2 = port.Port_BM_VaR(5,5/252, 0.99, 10000)
##-----
fig, ax = plt.subplots()
ax.plot(timeline, BM_port2[1:252*15],'r-', label='Port2 BM VaR 5 year')
legend = ax.legend(loc='upper right', shadow=True)
ax.set_title('Port2 BM VaR 5 year')
plt.show()

